In [ ]:
import sys

sys.path.append('../..')

In [ ]:
import warnings

warnings.filterwarnings('error')

In [ ]:
# Instead of taking 365 standard days or 252 trading days
# in order to get some easy computations for the eqty and df time grids
# I chose to take 360 days of tradings

step = 1/360.
mpor = 15*step

#Discount

In [ ]:
from finance.discountfactor import ConstantRateDiscountFactor 

maturity = 5.
r = 0.02

discount = ConstantRateDiscountFactor(r)

# Exponential distrib

In [ ]:
from scipy.stats import expon
 
mean = maturity / 2.
exp_distrib = expon(loc=0, scale=mean)

print "Mean = %s"%mean

# Underlying

In [ ]:
import numpy as np
from maths.montecarlo.processes.brownianmotions import BrownianMotion

time_grid = BrownianMotion.generate_time_grid(0, maturity, step)

x_0 = [100]
drift = 6
vol = 15

b = BrownianMotion(time_grid, x_0, drift, vol)

# Derivative

In [ ]:
from finance.products.european.assetswap import (
    SwapContract,
)

delta = 0.5

swap_dates = SwapContract.generate_payment_dates(0, maturity, delta)
swap = SwapContract(b, discount, swap_dates)

price_0 = swap.price(0.)

print swap
print "\nPrice swap at t=0 = ",price_0

# Exposure

In [ ]:
from risk.exposures import EuropeanQuantileBrownianExposure

quantile_im = 0.80

exposure = EuropeanQuantileBrownianExposure(swap, discount, drift, vol)

# Load intensities of obligors

In [ ]:
import dill

raw_intensities = None
with open('../intensities/intensities.pkl', 'rb') as f:
    raw_intensities = dill.load(f)
    
raw_intensities.keys()

In [ ]:
raw_idios = np.array(raw_intensities['idio'])
raw_groups = np.array(raw_intensities['groups'])

In [ ]:
subsets = []
hazard_rates = []
pillars = []

for d_idio in raw_idios:
    subsets.append(d_idio['id'])
    hazard_rates.append(d_idio['hazard_rates'])
    pillars.append(d_idio['pillars'])
    
for d_groups in raw_groups:    
    subsets.append(d_groups['ids'])
    hazard_rates.append(d_groups['hazard_rates'])
    pillars.append(d_groups['pillars'])

#Copula

In [ ]:
from maths.copula.marshallolkin import StepWiseIntensitiesMarshallOlkinCopula

copula = StepWiseIntensitiesMarshallOlkinCopula(subsets, hazard_rates, pillars)

# ID parameters

In [ ]:
bank_index = 113

counterparties_indexes = list(raw_groups[-2]['ids'])
counterparties_indexes.remove(bank_index)

bank_and_counterparties_indexes = list(raw_groups[-2]['ids'])

# Portfolio construction

In [ ]:
p_fixed = 100.
strike = swap.strike

delta_times = swap.delta_time
discount_factors = [discount(t) for t in swap.pillars[1:]]

delta_beta_sum = np.dot(delta_times, discount_factors)

notional = p_fixed / (strike*delta_beta_sum)

In [ ]:
from finance.portfolio import EquilibratedPortfolio

pos = EquilibratedPortfolio.generate_1_vs_all_positions(bank_index, bank_and_counterparties_indexes, len(raw_idios))
positions = np.array(pos*notional).reshape(pos.size, 1)

In [ ]:
port = EquilibratedPortfolio(positions, [swap], [exposure])

# VM and IM accounts

In [ ]:
from ccp.accounts import Accounts
from ccp.states import MembersState

derivatives_nb = 1

states = MembersState(raw_idios.size)
vm_accounts = Accounts(states, derivatives_nb)
im_accounts = Accounts(states, derivatives_nb)

In [ ]:
# Be carefull, here, as the default models will be 
# used for regulatory computations, we select only 
# the first 125 idiosyncratic models even if 
# we care only about the 61 first.

default_proba_models = copula.models[0 : raw_idios.size]

# Regulatory capital

In [ ]:
from risk.basel import RegulatoryCapital

recoveries = [0.4 for i in range(0, raw_idios.size)]

regul_cap = RegulatoryCapital(vm_accounts, im_accounts, port, default_proba_models, recoveries, bank_index=bank_index)

# Funding and capital parameters

In [ ]:
lambda_ = 0.
recov_funding = 1.
k_ = 0.1

c_0 = -.0020

#MC loop

In [ ]:
def time_offseter(time, time_grid_ref, left=False, max_value=1000):
    offset = -1 if left else 0
    idx = np.searchsorted(time_grid_ref, time, side='right') + offset
    
    if idx == len(time_grid):
        return max_value
    
    return time_grid_ref[idx]

In [ ]:
results_idio = {"cva": {c: {"sum": 0., "sum2": 0.} for c in counterparties_indexes},
                "dva": {c: {"sum": 0., "sum2": 0.} for c in counterparties_indexes},
                "fva": {c: {"sum": 0., "sum2": 0.} for c in counterparties_indexes},
                "kva_ccr": {c: {"sum": 0., "sum2": 0.} for c in counterparties_indexes},
                "kva_cva": {c: {"sum": 0., "sum2": 0.} for c in counterparties_indexes}}

results_globs = {"cva": {"sum": 0., "sum2": 0.},
                 "dva": {"sum": 0., "sum2": 0.},
                 "fva": {"sum": 0., "sum2": 0.},
                 "kva_ccr": {"sum": 0., "sum2": 0.},
                 "kva_cva": {"sum": 0., "sum2": 0.}}

In [ ]:
N = 10

## Saving default times and paths

In [ ]:
to_save = {}

## Subsets indexes of defaulters

In [ ]:
bank_and_counterparties_subsets_indexes = []
bank_subsets_indexes = None
for id_ in bank_and_counterparties_indexes:
    indexes = copula.get_indexes_including(id_)
    bank_and_counterparties_subsets_indexes += indexes
    
    if id_ == bank_index:
        bank_subsets_indexes = indexes
    
bank_and_counterparties_subsets_indexes = list(set(bank_and_counterparties_subsets_indexes))
bank_and_counterparties_subsets_indexes.sort()

# As we don't want to handle the armaggeddon case
# we must remove the last index (which is the armaggeddon)
bank_and_counterparties_subsets_indexes = bank_and_counterparties_subsets_indexes[:-1]
print bank_and_counterparties_subsets_indexes

to_save["defaulters_subsets_indexes"] = bank_and_counterparties_subsets_indexes

In [ ]:
import time

tic = time.time()

zetas = exp_distrib.rvs(size=N)
to_save["zetas"] = zetas

default_times_mat = copula.generate_default_times(subsets_indexes=bank_and_counterparties_subsets_indexes, number=N)
to_save["default_times"] = default_times_mat

In [ ]:
counterparties_subsets_indexes = bank_and_counterparties_subsets_indexes[:]
for ii in bank_subsets_indexes:
    try:
        counterparties_subsets_indexes.remove(ii)
    except Exception, e:
        bank_subsets_indexes.remove(ii)

In [ ]:
print counterparties_indexes, len(counterparties_indexes)
print bank_index
print
print counterparties_subsets_indexes, len(counterparties_subsets_indexes)
print bank_subsets_indexes, len(bank_subsets_indexes)

In [ ]:
to_save["underlying"] = {
    "time": [], 
    "values": []
}

In [ ]:
bk_default_times_indexes = [ii for ii, ind in enumerate(bank_and_counterparties_subsets_indexes) if ind in bank_subsets_indexes]

for i in range(N):
    # Resurrecting the states
    # reset to 0. the values
    # of the VMs and IMs
    states.resurrect_all()
    
    zeta = time_offseter(zetas[i], time_grid)
    zeta_delta = time_offseter(zeta+mpor, time_grid, True)
    
    keeping_pillars = swap.pillars[swap.pillars <= zeta_delta]
    step_time_grid = np.append(keeping_pillars, [zeta, zeta_delta])
    step_time_grid.sort()
    
    to_save["underlying"]["time"].append(step_time_grid)
    b.time = step_time_grid
    
    to_save["underlying"]["values"].append(b.values)
    
    default_times = default_times_mat[i]
    
    cva_, dva_, fva_, kccr_, kcva_ = [0. for _ in range(5)]
    
    for cp_index in counterparties_indexes:
        cp_subsets_indexes = copula.get_indexes_including(cp_index)        
        cp_default_times_indexes = [ii for ii, ind in enumerate(counterparties_subsets_indexes) if ind in cp_subsets_indexes]
        
        bk_cp_default_times_indexes = np.append(bk_default_times_indexes, cp_default_times_indexes)               
        bk_cp_default_times = default_times[bk_cp_default_times_indexes]
        
        tau = bk_cp_default_times.min()
        bar_tau = min(tau, maturity)
        
        cva__, dva__, fva__, kccr__, kcva__ = [0. for _ in range(5)]
        del bk_cp_default_times
        
        if bar_tau <= zeta:
            continue
            
        inv_pdf = 1./exp_distrib.pdf(zeta)
        discount_ratio = discount(zeta+mpor) / discount(zeta)
        mult_factor = inv_pdf*discount_ratio
        
        # IM computations
        b_im = port.compute_exposure(zeta, risk_period=mpor, conf_level=quantile_im, \
                                     from_=bank_index, towards_=cp_index)
        c_im = port.compute_exposure(zeta, risk_period=mpor, conf_level=quantile_im, \
                                     from_=cp_index, towards_=bank_index)
        
        im_accounts.put_amounts(bank_index, b_im)
        im_accounts.put_amounts(cp_index, c_im)
        
        d_pl = [s.price(zeta) for s in [swap]]
        b_vm = port.compute_value(d_pl, from_=bank_index, towards_=cp_index)
        vm_accounts.put_amounts(bank_index, b_vm)
        vm_accounts.put_amounts(cp_index, -b_vm)
        
        p_zeta = b_vm.sum()
        
        d_pl_zeta_delta = [s.price(zeta_delta) for s in [swap]]
        p_zeta_delta = port.compute_value(d_pl_zeta_delta, from_=bank_index, towards_=cp_index).sum()
        
        # We suppose here that \Delta_{\zeta} is null
        q_zeta_delta = p_zeta_delta
        
        b_default_times = default_times[bk_default_times_indexes]
        c_default_times = default_times[cp_default_times_indexes]
        b_min_default_time = b_default_times.min()
        c_min_default_time = c_default_times.min()
        
        for df_index in bk_cp_default_times_indexes:
            subset_index = bank_and_counterparties_subsets_indexes[df_index]
            gamma_zeta = copula.gamma(subset_index, zeta)
            
            ####################
            ##### CVA part #####
            ####################
            c_default_indic = df_index in cp_default_times_indexes
            if not c_default_indic:
                c_default_indic = c_min_default_time <= zeta_delta
                
            if c_default_indic:
                b_collat = p_zeta + im_accounts.get_amount(bank_index).sum()
                loss = np.maximum(q_zeta_delta-b_collat, 0.)
                cva__ += gamma_zeta*(1.-recoveries[cp_index]*loss)
                
            ####################
            ##### DVA part #####
            ####################
            b_default_indic = df_index in bk_default_times_indexes
            if not b_default_indic:
                b_default_indic = b_min_default_time <= zeta_delta
                
            if b_default_indic:
                c_collat = p_zeta - im_accounts.get_amount(cp_index).sum()
                loss = np.maximum(q_zeta_delta-c_collat, 0.)
                dva__ += gamma_zeta*(1.-recoveries[bank_index]*loss)
                
            ####################
            ##### FVA part #####
            ####################
            b_funded_collat = p_zeta + \
                                im_accounts.get_amount(bank_index).sum() - \
                                im_accounts.get_amount(cp_index).sum()

            b_gamma_zeta = copula.tot_gamma(bank_index, zeta)
            
            bar_lambda_zeta = .5*(1.-recoveries[bank_index])*b_gamma_zeta
            tilde_lambda_zeta = bar_lambda_zeta - (1.-recov_funding)*b_gamma_zeta
            c_zeta = c_0 + bar_lambda_zeta
            lambda_zeta = lambda_
            
            fva__ += b_gamma_zeta * (c_zeta*b_funded_collat + \
                                     tilde_lambda_zeta*np.maximum(p_zeta-b_funded_collat, 0.) + \
                                     lambda_zeta*np.minimum(p_zeta-b_funded_collat, 0.))
            
            ######################
            ##### K_CCR part #####
            ######################
            kccr__ += k_*regul_cap.compute_kccr(cp_index, zeta)
            
            ######################
            ##### K_CCR part #####
            ######################
            kcva__ += k_*regul_cap.compute_kcva(cp_index, zeta)
            
        results_idio['cva'][cp_index]["sum"] += (mult_factor*cva__)
        results_idio['cva'][cp_index]["sum2"] += (mult_factor*cva__)**2
        
        results_idio['dva'][cp_index]["sum"] += (mult_factor*dva__)
        results_idio['dva'][cp_index]["sum2"] += (mult_factor*dva__)**2
        
        results_idio['fva'][cp_index]["sum"] += (mult_factor*fva__)
        results_idio['fva'][cp_index]["sum2"] += (mult_factor*fva__)**2
        
        results_idio['kva_ccr'][cp_index]["sum"] += (mult_factor*kccr__)
        results_idio['kva_ccr'][cp_index]["sum2"] += (mult_factor*kccr__)**2
        
        results_idio['kva_cva'][cp_index]["sum"] += (mult_factor*kcva__)
        results_idio['kva_cva'][cp_index]["sum2"] += (mult_factor*kcva__)**2
        
        cva_ += (mult_factor*cva__)
        dva_ += (mult_factor*dva__)
        fva_ += (mult_factor*fva__)
        kccr_ += (mult_factor*kccr__)
        kcva_ += (mult_factor*kcva__)
    
    results_globs["cva"]["sum"] += cva_
    results_globs["cva"]["sum2"] += cva_**2
    
    results_globs["dva"]["sum"] += dva_
    results_globs["dva"]["sum2"] += dva_**2
    
    results_globs["fva"]["sum"] += fva_
    results_globs["fva"]["sum2"] += fva_**2
    
    results_globs["kva_ccr"]["sum"] += kccr_
    results_globs["kva_ccr"]["sum2"] += kccr_**2
    
    results_globs["kva_cva"]["sum"] += kcva_
    results_globs["kva_cva"]["sum2"] += kcva_**2
                    
toc = time.time()

#Results

In [ ]:
print "Results for %d iterations (%s secs.)"%(N, toc-tic)
print

print "Used discount factor: %s"%discount
print

print "Bank index %i that belongs to the following MO copula subsets:"%bank_index
for idx in bank_subsets_indexes:
    print "- %s with pillars %s and intensity %s\n"%([x for x in copula.subsets[idx][0]], copula.pillars[idx], copula.intensities[idx])
    
print "Counterparties indexes:"
print counterparties_indexes

print "\n-----------------------------------------------------\n"

for idx in counterparties_indexes:
    print "Counterparty index %s belongs to the following MO copula subses:"%idx
    sub_indexes = copula.get_indexes_including(idx)
    bank_subsets_indexes
    for subset_idx in sub_indexes:
        if subset_idx not in bank_subsets_indexes:
            print "- %s with pillars %s and intensity %s\n"%([x for x in copula.subsets[subset_idx][0]], copula.pillars[subset_idx], copula.intensities[subset_idx])        
    print
    
print "\n-----------------------------------------------------\n"
    
print "Derivatives:"
for d in [swap]:
    print "- %s"%d
    
print "\nPositions:"
print port.positions

print "Confidence level used for IM: %.2f"%quantile_im

In [ ]:
from scipy.stats import norm

conf_level = 0.95
z_level = norm.ppf(0.5*(1+conf_level))

print "Results of the individual xVA:\n"
keys = results_idio.keys()
keys.sort()

for k in keys:    
    print "%s:"%k.upper()
    print "---------------------------"
    v = results_idio[k]
    for id_, res_ in v.iteritems():        
        mean__ = res_['sum']/N
        mod_var__ = (res_['sum2']/N - mean__**2) / (N-1.)
        half_inter = z_level*np.sqrt(mod_var__)
        print "Id %s: [%f, %f]"%(id_, mean__-half_inter, mean__+half_inter)
    print

In [ ]:
print "Results of the global xVA:\n"
keys = results_globs.keys()
keys.sort()

for k in keys:
    mean_ = results_globs[k]['sum']/N
    mod_var_ = (results_globs[k]['sum2']/N - mean_**2) / (N-1.)
    half_inter = z_level*np.sqrt(mod_var_)
    
    print "The %s for the bank lies in [%f, %f]"%(k.upper(), mean_-half_inter, mean_+half_inter)    

In [ ]:
with open('../mc_simulated_data.pkl', 'wb') as f:
    dill.dump(to_save, f)